# **Importing Data**



In [ ]:
import os
import numpy as np
import glob
# import email
# import email.policy
import string
import re
import nltk
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

DATASETS_DIR = 'datasets'
MODELS_DIR = 'models'
TAR_DIR = os.path.join(DATASETS_DIR, 'tar')

# SPAM_URL = 'https://spamassassin.apache.org/old/publiccorpus/20021010_spam.tar.bz2'
SPAM2_URL = 'https://spamassassin.apache.org/old/publiccorpus/20050311_spam_2.tar.bz2'
# EASY_HAM_URL = 'https://spamassassin.apache.org/old/publiccorpus/20030228_easy_ham.tar.bz2'
EASY_HAM2_URL = 'https://spamassassin.apache.org/old/publiccorpus/20030228_easy_ham_2.tar.bz2'
HARD_HAM_URL = 'https://spamassassin.apache.org/old/publiccorpus/20030228_hard_ham.tar.bz2'

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from urllib.request import urlretrieve
import tarfile
import shutil

def download_dataset(url):
    """download and unzip data from a url into the specified path"""

    # create directory if it doesn't exist
    if not os.path.isdir(TAR_DIR):
        os.makedirs(TAR_DIR)

    filename = url.rsplit('/', 1)[-1]
    tarpath = os.path.join(TAR_DIR, filename)

    # download the tar file if it doesn't exist
    try:
        print("Downloading", tarpath)
        tarfile.open(tarpath)
    except:
        urlretrieve(url, tarpath)

    with tarfile.open(tarpath) as tar:
        dirname = os.path.join(DATASETS_DIR, tar.getnames()[0])
        if os.path.isdir(dirname):
            shutil.rmtree(dirname)
        tar.extractall(path=DATASETS_DIR)

        cmds_path = os.path.join(dirname, 'cmds')
        if os.path.isfile(cmds_path):
            os.remove(cmds_path)

    return dirname


def load_dataset(dirpath):
    """load emails from the specified directory"""

    files = []
    filepaths = glob.glob(dirpath + '/*')
    for path in filepaths:
        with open(path, 'rb') as f:
            byte_content = f.read()
            str_content = byte_content.decode('utf-8', errors='ignore')
            files.append(str_content)
            # files.append(email.parser.BytesParser(policy=email.policy.default).parse(f))

    return files


In [ ]:
# download the data

# spam_dir = download_dataset(SPAM_URL)
spam2_dir = download_dataset(SPAM2_URL)
# easy_ham_dir = download_dataset(EASY_HAM_URL)
easy_ham2_dir = download_dataset(EASY_HAM2_URL)
hard_ham_dir = download_dataset(HARD_HAM_URL)


# load the datasets from datasets/tar/*

# spam = load_dataset(spam_dir)
spam2 = load_dataset(spam2_dir)
# easy_ham = load_dataset(easy_ham_dir)
easy_ham2 = load_dataset(easy_ham2_dir)
hard_ham = load_dataset(hard_ham_dir)

print("Emails Ham :", len( easy_ham2 + hard_ham))
print("Emails Spam :" ,len(spam2))

Emails Ham : 1650
Emails Spam : 1396


In [ ]:
from  sklearn.utils import shuffle

# create the full dataset
X = spam2 + easy_ham2 + hard_ham
lables = np.concatenate((np.ones(len(spam2)), np.zeros(len(easy_ham2) + len(hard_ham))))

# shuffle the dataset
X, lables = shuffle(X, lables, random_state=42)

print("Data Impoted!!!!")

Data Impoted!!!!


# **Data Preparation**

In [40]:
# remove the head of eamil
def remove_header(email):
    """remove the header from an email"""
    return email[email.index('\n\n'):]

# vefify if the string is an url
# def is_url(s):
#     url = re.match("http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|"
#                      "[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", s)
#     return url is not None

# def remove_hyperlink(word):
#     return  re.sub(r"http\S+", "URL", word)


def remove_hyperlink(word):
    regex_links = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    word_without_links =  re.sub(regex_links,"", word)
    regex_email = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,7}\b'
    return re.sub(regex_email,"", word_without_links)




# make word in lower case
def to_lower(word):
    return word.lower()

# replace URL with URL word
# def url_to_word(words):
#     """convert all urls in the list to the word 'URL'"""
#     for i, word in enumerate(words):
#         if is_url(word):
#             words[i] = 'URL'
#     return " ".join(words)

def remove_whitespace(word):
    return re.sub('\s+', '', word)

def remove_digits(word):
  '''This function removes all the numbers'''
  return re.sub('\d+', '', word)

def remove_underscores(word):
  '''This function removes all the underscores'''
  return re.sub(r'_', '', word)

def remove_excess_whitespace(word):
  '''This function removes excess white spaces'''
  return re.sub('\s+', '', word)

def remove_special_characters(word):
  '''This function removes all the special characters'''
  return re.sub('\W', '', word)

# remove stop words
stopwords_english = stopwords.words('english')

# def remove_stop_words(words,stopwords_list=stopwords_english):
#     for i in words:
#       if i not in  stopwords_list:
#         return i
def remove_stopwords(word,stopword_list=stopwords_english):
  '''This function removes the stop words'''
  word_list = word.split(" ")
  cleaned_txt = [w for w in word_list if not w in stopword_list]
  cleaned_string = " ".join(cleaned_txt)

  return cleaned_string

print("Preprocessing functions readyy!")

Preprocessing functions readyy!


In [ ]:
print(X[0])

Received: from hq.pro-ns.net (localhost [127.0.0.1])
	by hq.pro-ns.net (8.12.5/8.12.5) with ESMTP id g6NEkmhY030813
	(version=TLSv1/SSLv3 cipher=EDH-DSS-DES-CBC3-SHA bits=168 verify=NO)
	for <cypherpunks-forward@ds.pro-ns.net>; Tue, 23 Jul 2002 09:46:48 -0500 (CDT)
	(envelope-from cpunks@hq.pro-ns.net)
Received: (from cpunks@localhost)
	by hq.pro-ns.net (8.12.5/8.12.5/Submit) id g6NEklSB030808
	for cypherpunks-forward@ds.pro-ns.net; Tue, 23 Jul 2002 09:46:47 -0500 (CDT)
Received: from einstein.ssz.com (cpunks@[207.200.56.4])
	by hq.pro-ns.net (8.12.5/8.12.5) with ESMTP id g6NEfuhX028821
	for <cypherpunks@ds.pro-ns.net>; Tue, 23 Jul 2002 09:41:56 -0500 (CDT)
	(envelope-from cpunks@einstein.ssz.com)
X-Authentication-Warning: hq.pro-ns.net: Host cpunks@[207.200.56.4] claimed to be einstein.ssz.com
Received: (from cpunks@localhost)
	by einstein.ssz.com (8.8.8/8.8.8) id JAA13628
	for cypherpunks@ds.pro-ns.net; Tue, 23 Jul 2002 09:50:29 -0500
Received: (from mdom@localhost)
	by einstein.ssz.

In [ ]:
# remove head
email_without_head = remove_header(X[0])
print(email_without_head)




<> Get up to 4 receivers installed in 4 rooms! 
<> No Equipment To Buy. 
<> Fist 3 Months of Free Service! - 
<> Up to 170 CHANNELS of CD quality sound and picture 
<> PROGRAMMING LESS EXPENSIVE than cable TV in most markets 

You can receive FREE INSTALLATION of a Dish Network 
Satellite TV System! You can also upgrade to a Personal Digital Video Recorder. 
(Retail value $499 if you had to buy this!) 

Click here to get your FREE INSTALLATION of a Dish Network satellite TV 
System and 3 months of free service before this promotion expires: 

http://www.proleadcom.com/discsat/index.php


--DeathToSpamDeathToSpamDeathToSpam--


-------------------------------------------------------
This sf.net email is sponsored by:ThinkGeek
Welcome to geek heaven.
http://thinkgeek.com/sf
_______________________________________________
Spamassassin-Sightings mailing list
Spamassassin-Sightings@lists.sourceforge.net
https://lists.sourceforge.net/lists/listinfo/spamassassin-sightings





In [ ]:
# to lower
email_lower = to_lower(email_without_head)
print(email_lower)




<> get up to 4 receivers installed in 4 rooms! 
<> no equipment to buy. 
<> fist 3 months of free service! - 
<> up to 170 channels of cd quality sound and picture 
<> programming less expensive than cable tv in most markets 

you can receive free installation of a dish network 
satellite tv system! you can also upgrade to a personal digital video recorder. 
(retail value $499 if you had to buy this!) 

click here to get your free installation of a dish network satellite tv 
system and 3 months of free service before this promotion expires: 

http://www.proleadcom.com/discsat/index.php


--deathtospamdeathtospamdeathtospam--


-------------------------------------------------------
this sf.net email is sponsored by:thinkgeek
welcome to geek heaven.
http://thinkgeek.com/sf
_______________________________________________
spamassassin-sightings mailing list
spamassassin-sightings@lists.sourceforge.net
https://lists.sourceforge.net/lists/listinfo/spamassassin-sightings





In [41]:
# replace URLs with URL word
email_without_urls = remove_hyperlink(email_lower)
print(email_without_urls)




<> get up to 4 receivers installed in 4 rooms! 
<> no equipment to buy. 
<> fist 3 months of free service! - 
<> up to 170 channels of cd quality sound and picture 
<> programming less expensive than cable tv in most markets 

you can receive free installation of a dish network 
satellite tv system! you can also upgrade to a personal digital video recorder. 
(retail value $499 if you had to buy this!) 

click here to get your free installation of a dish network satellite tv 
system and 3 months of free service before this promotion expires: 




--deathtospamdeathtospamdeathtospam--


-------------------------------------------------------
this sf.net email is sponsored by:thinkgeek
welcome to geek heaven.

_______________________________________________
spamassassin-sightings mailing list







**Pipline**

In [ ]:
def EmailsPreprocessor(sentence):
    Preprocessor_utils = [remove_header,
                      to_lower,
                      url_to_word,
                      remove_whitespace,
                      remove_digits,
                      remove_underscores,
                      remove_excess_whitespace,
                      remove_special_characters,
                      remove_stopwords]

    for tool in Preprocessor_utils:
        sentence = tool(sentence)
        # print(sentence)
    return sentence

X_features = [EmailsPreprocessor(email) for email in X]
X_features[0]

'     g e u p r e c e v e r n l l e n r       n e q u p e n b u       f n h f f r e e e r v c e         u p c h n n e l f c q u l u n n p c u r e     p r g r n g l e e x p e n v e h n c b l e v n r k e u c n r e c e v e f r e e n l l n f h n e w r k e l l e v e   u c n l u p g r e p e r n l g l v e r e c r e r     r e l v l u e   f u h b u h     c l c k h e r e g e u r f r e e n l l n f h n e w r k e l l e v e n n h f f r e e e r v c e b e f r e h p r n e x p r e   h p       w w w   p r l e c   c   c   n e x   p h p     e h p e h p e h p                                                                                                                   h f   n e e l p n r e b   h n k g e e k w e l c e g e e k h e v e n   h p       h n k g e e k   c   f p n   g h n g l n g l p n   g h n g   l   u r c e f r g e   n e h p       l   u r c e f r g e   n e   l   l n f   p n   g h n g '

In [ ]:
from sklearn.model_selection import train_test_split

# split the data into stratified training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_features, lables, test_size=0.2, stratify=y,
                                                    random_state=42)
print("X_train shape : ",len(X_train))
print("y_train shape : ",len(y_train))
print("X_test shape : ",len(X_test))
print("y_test shape : ",len(y_test))

X_train shape :  2436
y_train shape :  2436
X_test shape :  610
y_test shape :  610


In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class EmailsPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self, no_header=True, to_lowercase=True, url_to_word=True, num_to_word=True,
                 remove_punc=True):
        self.no_header = no_header
        self.to_lowercase = to_lowercase
        self.url_to_word = url_to_word
        self.num_to_word = num_to_word
        self.remove_punc = remove_punc
        # self.stemming = stemming

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X_cleaned = []
        for email in X:
            if self.no_header:
                email = remove_header(email)
            if self.to_lowercase:
                email = email.lower()

            email_words = email.split()
            if self.url_to_word:
                email_words = convert_url_to_word(email_words)
            if self.num_to_word:
                email_words = convert_num_to_word(email_words)
            email = ' '.join(email_words)
            if self.remove_punc:
                email = remove_punctuation(email)
            # if self.stemming:
            #     email = remove_punctuation(email)

            X_cleaned.append(email)
            print(X_cleaned)

        return X_cleaned


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer

# full preparation pipeline
prepare_pipeline = Pipeline([
    ('clean_email', EmailsPreprocessor()),
    ('bag_of_words', CountVectorizer())
])

# prepare the training data
X_train_prepared = prepare_pipeline.fit_transform([X_train[0]])

['this is an html email message if you see this your mail client does not support html messages nextpartaaxpmnbmet contenttype texthtmlcharsetiso88591 contenttransferencoding 7bit htmlheadtitle75 reductiontitlehead body body bgcolorpink pdiv alignrightfont colorbluebu august2002bufontdivpbr br pfont facetimes new romanstrong dear sir madamfontstrongpbr pfont facetimes new roman bin case you have received this mail earlier kindly ignore this mail it may have been resent to you by mistake ours is not a mailing list and we have no intention of sending any regular mails to anybodybfontp brpfont facetimes new roman strongbrpfont facearial strong font colorredc current yearly automobile sales worldwide  more than NUM billion us strongfontp we have devised a set of systems which can reduce the damage caused to vehicles and deaths and injury caused to passengers and pedestrians in vehicular accidents by more thanstrongfont font colorredbuseventyfive percentbufontpbr pfont facetimes new roman s

In [ ]:
print(X_train[0])

From fork-admin@xent.com  Fri Aug  2 11:28:34 2002
Return-Path: <fork-admin@xent.com>
Delivered-To: yyyy@localhost.netnoteinc.com
Received: from localhost (localhost [127.0.0.1])
	by phobos.labs.netnoteinc.com (Postfix) with ESMTP id 6E07E440F7
	for <jm@localhost>; Fri,  2 Aug 2002 06:28:29 -0400 (EDT)
Received: from phobos [127.0.0.1]
	by localhost with IMAP (fetchmail-5.9.0)
	for jm@localhost (single-drop); Fri, 02 Aug 2002 11:28:29 +0100 (IST)
Received: from xent.com ([64.161.22.236]) by dogma.slashnull.org
    (8.11.6/8.11.6) with ESMTP id g72ARd216304 for <jm@jmason.org>;
    Fri, 2 Aug 2002 11:27:43 +0100
Received: from lair.xent.com (localhost [127.0.0.1]) by xent.com (Postfix)
    with ESMTP id 0279329409F; Fri,  2 Aug 2002 03:25:05 -0700 (PDT)
Delivered-To: fork@spamassassin.taint.org
Received: from mail.xent.com (unknown [202.54.79.189]) by xent.com
    (Postfix) with SMTP id 3E101294099 for <fork@xent.com>; Fri,
    2 Aug 2002 03:23:56 -0700 (PDT)
From: "abc" <largeinv@india

# **Load Data**

In [ ]:
!unzip /content/emails_datasets.zip

Archive:  /content/emails_datasets.zip
  inflating: spam_assassin.csv       


In [ ]:
import pandas as pd

data_email = pd.read_csv("spam_assassin.csv")
data_email.head(20)

,text,target
0,From ilug-admin@linux.ie Mon Jul 29 11:28:02 2...,0
1,From gort44@excite.com Mon Jun 24 17:54:21 200...,1
2,From fork-admin@xent.com Mon Jul 29 11:39:57 2...,1
3,From dcm123@btamail.net.cn Mon Jun 24 17:49:23...,1
4,From ilug-admin@linux.ie Mon Aug 19 11:02:47 2...,0
5,From tobaccodemon@terra.es Sat Sep 7 22:05:58 ...,1
6,From larlar78@MailOps.Com Sat Jun 30 00:19:08 ...,1
7,From rpm-list-admin@freshrpms.net Thu Jul 25 1...,0
8,From exmh-users-admin@redhat.com Wed Aug 7 06:...,0
9,From contractor@goldenbay.com.cy Tue Jul 23 23...,1


In [ ]:
data_email.text.iloc[0]

"From ilug-admin@linux.ie Mon Jul 29 11:28:02 2002 Return-Path: <ilug-admin@linux.ie> Delivered-To: yyyy@localhost.netnoteinc.com Received: from localhost (localhost [127.0.0.1]) by phobos.labs.netnoteinc.com (Postfix) with ESMTP id A13D94414F for <jm@localhost>; Mon, 29 Jul 2002 06:25:11 -0400 (EDT) Received: from phobos [127.0.0.1] by localhost with IMAP (fetchmail-5.9.0) for jm@localhost (single-drop); Mon, 29 Jul 2002 11:25:11 +0100 (IST) Received: from lugh.tuatha.org (root@lugh.tuatha.org [194.125.145.45]) by dogma.slashnull.org (8.11.6/8.11.6) with ESMTP id g6RHn7i17130 for <jm-ilug@jmason.org>; Sat, 27 Jul 2002 18:49:07 +0100 Received: from lugh (root@localhost [127.0.0.1]) by lugh.tuatha.org (8.9.3/8.9.3) with ESMTP id SAA25016; Sat, 27 Jul 2002 18:45:03 +0100 X-Authentication-Warning: lugh.tuatha.org: Host root@localhost [127.0.0.1] claimed to be lugh Received: from mail1.mail.iol.ie (mail1.mail.iol.ie [194.125.2.192]) by lugh.tuatha.org (8.9.3/8.9.3) with ESMTP id SAA24977 f

In [ ]:
data_email.describe()

,target
count,5796.000000
mean,0.327122
std,0.469203
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [ ]:
data_email.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5796 entries, 0 to 5795
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    5796 non-null   object
 1   target  5796 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 90.7+ KB
